In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# data
leases = pd.read_csv('Leases.csv')
prices = pd.read_csv('Price and Availability Data.csv')
markets = ['Atlanta', 'Austin', 'Baltimore', 'Boston', 'Charlotte', 'Chicago', 'Chicago Suburbs', 'Dallas/Ft Worth', 'Denver', 'Detroit', 'Houston', 'Los Angeles', 'Manhattan', 'Nashville', 'Northern New Jersey', 'Northern Virginia', 'Orange County', 'Philadelphia', 'Phoenix', 'Raleigh/Durham', 'Salt Lake City', 'San Diego', 'San Francisco', 'Seattle', 'South Bay/San Jose', 'South Florida', 'Maryland', 'Tampa', 'Washington D.C.']

In [3]:
# edit columns
market_mapping = {
    'Downtown Chicago': 'Chicago',
    'Denver-Boulder': 'Denver',
    'Suburban Maryland': 'Maryland',
    'Raleigh-Durham': 'Raleigh/Durham',
    'Dallas-Ft. Worth': 'Dallas/Ft Worth',
    'Orange County (CA)': 'Orange County',
    'South Bay': 'South Bay/San Jose',
    'Washington DC': 'Washington D.C.'
}
prices['market'] = prices['market'].replace(market_mapping)

In [6]:
best_quarters_by_market = {}
for market in markets:
    best_quarter = find_best_quarter(market)
    best_quarters_by_market[market] = best_quarter

In [7]:
best_quarters_by_market

{'Atlanta': 'Q1',
 'Austin': 'Q3',
 'Baltimore': 'Q2',
 'Boston': 'Q1',
 'Charlotte': 'Q1',
 'Chicago': 'Q1',
 'Chicago Suburbs': 'Q3',
 'Dallas/Ft Worth': 'Q3',
 'Denver': 'Q1',
 'Detroit': 'Q3',
 'Houston': 'Q1',
 'Los Angeles': 'Q1',
 'Manhattan': 'Q3',
 'Nashville': 'Q1',
 'Northern New Jersey': 'Q1',
 'Northern Virginia': 'Q1',
 'Orange County': 'Q1',
 'Philadelphia': 'Q1',
 'Phoenix': 'Q1',
 'Raleigh/Durham': 'Q1',
 'Salt Lake City': 'Q1',
 'San Diego': 'Q3',
 'San Francisco': 'Q4',
 'Seattle': 'Q3',
 'South Bay/San Jose': 'Q1',
 'South Florida': 'Q1',
 'Maryland': 'Q1',
 'Tampa': 'Q1',
 'Washington D.C.': 'Q1'}

In [5]:
def find_best_quarter(market):
    market_data = prices[prices['market'] == market].copy()

    columns_to_keep = ['market', 'year', 'quarter', 'internal_class_rent', 'availability_proportion']
    market_data = market_data[columns_to_keep].copy()
    
    market_data = market_data.groupby(['year', 'quarter']).agg({
        'internal_class_rent': 'sum',
        'availability_proportion': 'mean'
    }).reset_index()
    
    normalized_rent = normalize_lower_better(market_data['internal_class_rent'])
    normalized_availability = normalize_higher_better(market_data['availability_proportion']) 
    
    market_data['score'] = normalized_rent * 0.7 + normalized_availability * 0.3
    market_data['score'] = market_data['score'].round(4)
    
    quarter_scores = market_data.groupby('quarter')['score'].mean().reset_index()
    best_quarter = quarter_scores.loc[quarter_scores['score'].idxmax()]
    
    return best_quarter[0]

In [4]:
# Function to normalize values (where lower is better)
def normalize_lower_better(series):
    min_val = series.min()
    max_val = series.max()
    range_val = max_val - min_val
    if range_val == 0:
        return pd.Series(0.5, index=series.index)  # If all values are the same
    return 1 - ((series - min_val) / range_val)  # Invert so lower values get higher scores

# Function to normalize values (where higher is better)
def normalize_higher_better(series):
    min_val = series.min()
    max_val = series.max()
    range_val = max_val - min_val
    if range_val == 0:
        return pd.Series(0.5, index=series.index)  # If all values are the same
    return (series - min_val) / range_val  # Higher values get higher scores